In [1]:
import sys

## This to get the peerless target star DataFrame for example purposes
#sys.path.append('/u/tdm/repositories/peerless/prediction')
#sys.path.append('/u/tdm/repositories/peerless')
#from targets import targets


import pandas as pd
targets = pd.read_hdf('targets.h5')

# The action is here. Depends on vespa & isochrones.
from exosyspop.populations import KeplerBinaryPopulation

In [2]:
pop = KeplerBinaryPopulation(targets, fB=0.4)

In [3]:
# Accessing secondary properties will initialize a secondary simulation,
# calling pop._generate_binaries().  The first time this is called, the
# secondary property regressors get trained.
pop.radius_B

dmag regressor trained, R2=0.999518867152
qR regressor trained, R2=0.999517339583


array([        nan,  0.96345122,         nan, ...,         nan,
        0.4876422 ,  0.66056978])

In [4]:
# subsequent calls are much faster; e.g.
pop._generate_binaries()
print(pop.radius_B)
%timeit pop._generate_binaries()

[       nan        nan  0.7760728 ...,        nan        nan        nan]
10 loops, best of 3: 99.6 ms per loop


In [5]:
# If physical accuracy is important, you can also choose to generate binary properties
# directly from the isochrone, but it's a factor of a few slower:
pop._generate_binaries(use_ic=True)
print(pop.radius_B)
%timeit pop._generate_binaries(use_ic=True)

[        nan         nan         nan ...,         nan  0.25332     0.52269921]
1 loops, best of 3: 363 ms per loop


In [6]:
# Similarly, accessing orbital properties will generate them
pop.period

array([  2.70754341e+02,   1.50263650e+03,   1.59658107e+09, ...,
         3.70762435e+01,   3.38338829e+05,   1.81463417e+06])

# Synthetic observations

In [7]:
# Now, we can observe and see what we see.  This takes into account
# duty cycle & data span, as well as geometry.
obs = pop.observe()
print(len(obs))
print(obs.columns)
obs.head()

543
Index([u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri', u'b_sec', u'k',
       u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri', u'T14_sec', u'T23_pri',
       u'T23_sec', u'dataspan', u'dutycycle', u'flux_ratio', u'n_pri',
       u'n_sec', u'phase_sec'],
      dtype='object')


,period,ecc,w,inc,a,aR,b_pri,b_sec,k,tra,...,T14_pri,T14_sec,T23_pri,T23_sec,dataspan,dutycycle,flux_ratio,n_pri,n_sec,phase_sec
297,1.972736,0.337185,0.800068,1.337084,4.868477e+11,8.939893,1.477581,2.420525,0.482794,True,...,0.006800,0.000000,0.000000,0.000000,1459.789,0.8751,0.025788,648,0,0.652602
777,26.807846,0.412153,3.814548,1.538211,3.322826e+12,42.088512,1.531805,0.905638,0.844951,True,...,0.255781,0.236416,0.000000,0.000000,1459.789,0.6987,0.587086,38,33,0.291698
1773,32.332283,0.162323,2.090950,1.564085,3.815206e+12,38.839093,0.222459,0.295402,0.657896,True,...,0.376652,0.496640,0.059567,0.052514,1459.789,0.6986,0.319264,28,33,0.448180
2484,1.385138,0.494728,1.674322,1.153620,3.938570e+11,6.588954,1.351326,3.969690,0.383128,True,...,0.012576,0.000000,0.000000,0.000000,1459.789,0.8752,0.009414,927,0,0.462634
2571,9.281982,0.417876,2.192676,1.531918,1.441547e+12,23.363715,0.559503,1.135045,0.455617,True,...,0.115395,0.158710,0.000000,0.000000,1459.789,0.8749,0.014301,134,136,0.337085


In [8]:
# This is pretty fast, even when generating a new population each time:
%timeit pop.observe(new=True)

1 loops, best of 3: 223 ms per loop


In [9]:
# Even faster if we only generate new orbits.
%timeit pop.observe(new_orbits=True)

10 loops, best of 3: 91.9 ms per loop


In [10]:
# So we can predict the expected number of observations pretty easily.
import numpy as np
N = 100
n_obs = np.array([len(pop.observe(new_orbits=True)) for i in range(N)])
n_obs.mean(), n_obs.std()

(534.16999999999996, 20.439204974753785)

In [11]:
# Notice that the above does not yet have trapezoidal parameters.  There are two options to generate these.
# Either we can set the fit_trap parameter, as follows:
obs = pop.observe(fit_trap=True)
print(len(obs))
obs.columns

1176


Index([u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri', u'b_sec', u'k',
       u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri', u'T14_sec', u'T23_pri',
       u'T23_sec', u'dataspan', u'dutycycle', u'flux_ratio', u'n_pri',
       u'n_sec', u'phase_sec', u'trap_dur_pri', u'trap_depth_pri',
       u'trap_slope_pri', u'trap_dur_sec', u'trap_depth_sec',
       u'trap_slope_sec'],
      dtype='object')

In [12]:
# All things considered, this is still pretty fast if we just need to do it a few times:
%timeit pop.observe(fit_trap=True)

1 loops, best of 3: 2.86 s per loop


In [13]:
# However, this is pretty slow if we want to do inference.  To help with this, we can 
# tell it to train & use a regression.  Training only happens once; by default with 10,000 
# synthetic observations.  This takes a minute or so.
obs = pop.observe(regr_trap=True)
print(len(obs))
obs.columns

Depth trained: R2=0.778021712264
Duration trained: R2=0.797347212778
Slope trained: R2=0.839444162163
511


Index([u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri', u'b_sec', u'k',
       u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri', u'T14_sec', u'T23_pri',
       u'T23_sec', u'dataspan', u'dutycycle', u'flux_ratio', u'n_pri',
       u'n_sec', u'phase_sec', u'trap_dur_pri_regr', u'trap_depth_pri_regr',
       u'trap_slope_pri_regr', u'trap_dur_sec_regr', u'trap_depth_sec_regr',
       u'trap_slope_sec_regr'],
      dtype='object')

In [14]:
# Subsequent calls are much faster
%timeit pop.observe(regr_trap=True)

10 loops, best of 3: 24.8 ms per loop


In [15]:
# Even generating a new stellar population & observing it is pretty quick
%timeit pop.observe(regr_trap=True, new=True)

1 loops, best of 3: 238 ms per loop


In [16]:
# Or again, you can just generate new orbits (rather than new binaries & new orbits)
%timeit pop.observe(regr_trap=True, new_orbits=True)

10 loops, best of 3: 105 ms per loop


In [17]:
# Generating the training data used for the trapezoid shape regression above used
# this function, which can be otherwise useful to sample >N random observations 
# from the existing population.  `trap_regr` defaults to `True` here.  
# This function also takes `new` or `new_orbits` keywords.
obs_pop = pop.get_N_observed(N=10000, new_orbits=True)
print(len(obs_pop))
obs_pop.columns

10166


Index([u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri', u'b_sec', u'k',
       u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri', u'T14_sec', u'T23_pri',
       u'T23_sec', u'dataspan', u'dutycycle', u'flux_ratio', u'n_pri',
       u'n_sec', u'phase_sec', u'trap_dur_pri_regr', u'trap_depth_pri_regr',
       u'trap_slope_pri_regr', u'trap_dur_sec_regr', u'trap_depth_sec_regr',
       u'trap_slope_sec_regr'],
      dtype='object')

In [18]:
# We can now look, e.g. at the expected number of single/double eclipsing systems:
query = '(n_pri < 3) & (n_sec < 3) & (n_pri==0 | n_sec==0)'
N = 100
n_obs = np.array([len(pop.observe(new_orbits=True).query(query)) for i in range(N)])
n_obs.mean(), n_obs.std()

(6.0099999999999998, 2.5514505678143169)

In [19]:
# Try this again, this time using the empirical eccentricity distribution
# (as opposed to the beta distribution with default params)---eccentricity matters!
pop.ecc_empirical = True
n_obs = np.array([len(pop.observe(new_orbits=True).query(query)) for i in range(N)])
n_obs.mean(), n_obs.std()

(10.5, 3.3926390907374748)